In [1]:
import ngsolve as ngs
from do import solve
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from copy import deepcopy

from createGeom import makeGeo
geoOCC = makeGeo()

ngmesh = geoOCC.GenerateMesh()
mesh = ngs.Mesh(ngmesh)

levels = 2
deg = 2

In [2]:
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

strom = 1e5
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')
J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
# Draw(J, mesh)

BH data from TEAM 13 problem
Applying a current of 196.88 Amps


In [3]:
# from do import solve

# ngmesh = geoOCC.GenerateMesh()
# mesh = ngs.Mesh(ngmesh)

HCurl = ngs.HCurl(mesh, order = deg, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face', autoupdate = True)
# HCurlN = ngs.HCurl(mesh, order = deg)
V3L2 = ngs.L2(mesh, dim=3, order = deg-1, autoupdate = True)
# # V3L2 = ngs.L2(mesh, dim=3, order = deg-1)
# VL2 = ngs.L2(mesh, order = deg-1)


# J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
# A0 = ngs.GridFunction(HCurl)
# A0, it0 = solve(HCurl,A0,mesh,deg,J,fun_w,fun_dw,fun_ddw)

In [4]:
# ngmesh = geoOCC.GenerateMesh()
# mesh = ngs.Mesh(ngmesh)

A0 = ngs.GridFunction(HCurl)
A0, it0 = solve(HCurl,A0,mesh,deg,J,fun_w,fun_dw,fun_ddw)


B0 = ngs.GridFunction(V3L2, nested = True)
with ngs.TaskManager(): B0.Set(ngs.curl(A0), ngs.VOL)

# with ngs.TaskManager(): B0.Set(ngs.CF((ngs.x,ngs.y,0)))
# a = deepcopy(B0.vec.data.FV().NumPy())
# plt.plot(a)
# B0.vec.data = a

# print(a.min(),a.max())

# mesh.Refine()
# B0.Update

clipping = { "function" : True, "z" : -0.4}
# with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)
with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)

# -8.468579201651239e-05 8.738808586428822e-05


Using 3D mesh with ne= 1057 elements and nv= 307 points and  6242 DOFs.
 


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [5]:
# print(B0.vec.data.FV().NumPy().size, 3*mesh.nface)
# B0

In [8]:
mesh.Refine()

V3L2_2 = ngs.L2(mesh, dim=3, order = deg-1)
B01 = ngs.GridFunction(V3L2_2)
B01.Set(B0, ngs.VOL)

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B01, mesh, clipping = clipping)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [7]:
clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…